# Machine Learning Demonstrations

In this notebook, we demonstrate how to use the Kx kdb+/q Automated Machine Learning library. The example below uses samples from the Telco Customer Churn dataset and IMBD movie review dataset.

<div class="alert alert-danger">
To run the below notebook, ensure that dependencies specified in <b>requirements.txt</b> have been correctly installed.
</div>

## Machine Learning Toolkit

The Machine Learning Toolkit ([ML-Toolkit](https://github.com/KxSystems/ml)) contains general use utilities, an implementation of the FRESH (Feature Extraction based on Scalable Hypothesis tests) algorithm, cross validation functions, clustering libraries and time series functionality. The primary purpose of these libraries are to provide kdb+/q users with access to commonly-used ML functions for preprocessing data, extracting features and scoring results.

## Automated Machine Learning in kdb+/q

The kdb+/q [Automated Machine Learning](https://github.com/kxsystems/automl/) library is built largely on the tools available within the kdb+/q ML-Toolkit. The purpose of this library is to provide users with the ability to automate the process of applying machine learning techniques to real-world problems. In the absence of expert machine learning engineers this framework handles the following processes within a traditional workflow:

- Data preprocessing
- Feature engineering and feature selection
- Model selection
- Hyperparameter tuning
- Report generation and model persistence

Each of these steps is outlined in depth within the documentation for this platform [here](https://code.kx.com/q/ml/automl). This allows users to understand the processes by which decisions are being made and the transformations which their data undergo during the production of the output models.

At present the supported machine learning problem types for classification and regression tasks and based on:

- One-to-one feature to target non time-series
- FRESH based feature extraction and model production
- NLP-based feature creation and word2vec transformation

The problems which can be solved by this framework will be expanded over time as will the available functionality.

### Multi-processing

This library supports multi-processed grid-search/cross-validation procedures and FRESH feature creation provided a user set `-s -8` in the JUPYTERQ_SERVERARGS entry to the appropriate JSON file, instructions to facilitate this can be found [here](https://code.kx.com/q/ml/jupyterq/notebooks/#server-command-line-arguments). In this demo, we use 8 worker processes and open a centralised port as below.

In [ ]:
\p 5124

The Kx Automated Machine Learning library is then loaded in as follows:

In [ ]:
// load in automl
\l automl/automl.q
.automl.loadfile`:init.q

// load utils
\l ../utils/util.q
\l ../utils/graphics.q

---

## Contents

Below, the notebook has been split into 3 distinct sections:
1. [Default Configurations](#Default-Configurations)
2. [Default NLP Configurations](#Default-NLP-Configurations)
3. [Custom Configurations (Advanced)](#Custom-Configurations-(Advanced))

---

### Telco Customer Churn

The [Telco Customer Churn dataset](https://www.kaggle.com/blastchar/telco-customer-churn/data) contains the following information.
* Data on 7,043 customers of a telecom provider provided by IBM
* Customer feature information including
    * What form of internet the user has (DSL/Fiber Optic)?
    * What are the users monthly payments?
    * How long has the customer been in their contract?
    * What services does the customer use? i.e. phone, internet, online backup, streaming etc.
* A target variable 'Churn' indicating if a user has cancelled their contract in the last month


In each case of the examples below, we aim to create a model which can accurately predict customer churn based on 20 features relating to each customer.

### Load data

In [ ]:
\S 42

// load data and separate into features and target
telcoData:5000?("SSBSSISSSSSSSSSSSSFFS";(),",")0:`:../data/telco.csv
telcoTarg:(`No`Yes!0 1)telcoData`Churn
telcoFeat:delete Churn from telcoData

// inspect data
-1"Shape of feature data is: ",(" x "sv string .ml.shape telcoFeat),"\n";
show 5#telcoFeat
-1"\nDistribution of target values:\n";
show update pcnt:.util.round[;.01]100*num%sum num from select num:count i by target from([]target:telcoTarg);

### Prepare data

In order to test both the model generation and prediction steps of the workflow we split the dataset into a training and testing set where

| Dataset form | Purpose                                                            | Percentage (%)|
|--------------|:-------------------------------------------------------------------|---------------|
| Training     | Generate model for deployment using `.automl.fit`                  | 90            |
| Testing      | Independent dataset to test application of `predict` functionality | 10            |

__*Note:*__ 

    We have set a random seed so that results can be replicated.

In [ ]:
\S 42
show telcoInputs:.ml.traintestsplit[telcoFeat;telcoTarg;.1]

## Default Configurations

### Inputs

The automated machine learning pipeline will use the training features (`xtrain`) and targets (`ytrain`) from `telcoInputs` above as input to `automl.fit`. 

Appropriate preprocessing steps including feature creation and selection will be applied to the data before being passed to a variety of machine learning models, choosing the best performing model. 

In this case, we select ``` `normal``` feature extraction as we have a 1-to-1 mapping between features and targets. We also use ``` `class``` for the problem type as we are dealing with a binary classification problem.

**Inportant:** 

    For the purposes of this demonstration we will pass in a dictionary in place of the default parameter (::). In order to ensure replication for users of this notebook the random seed parameter `seed is set in this example with the remaining parameters defaulted.

In [ ]:
telcoFeats  :telcoInputs`xtrain       / features
telcoTarget :telcoInputs`ytrain       / targets
featureType1:`normal                  / normal feature extraction
problemType1:`class                   / classification problem
paramDict1  :enlist[`seed]!enlist 350 / default configuration

In the default configuration, information generated during the fitting of the model will we saved to the outputs folder. This includes metadata information, graphs, reports and the fit model.

In addition to saving outputs, the function returns a dictionary with two keys:

 Return key  | Description
-------------|:-------------
 `modelInfo` | Metadata information generated from the pipeline such as preprocessing steps taken, significant features chosen and any other information needed to replicate the results.
 `predict`   | A function containing all relevant information and procedures required to generate new predictions using the fit model

We can now run `.automl.fit` using the default setting with out training set from the Telco Customer Churn dataset.

In [ ]:
start:.z.t
model1:.automl.fit[telcoFeats;telcoTarget;featureType1;problemType1;paramDict1]
-1"\n.automl.fit took ",string .z.t-start;
-1"\nReturn of .automl.fit:\n",.Q.s[model1];

#### Feature impact

<img src="images/run1impact.png" />

We see that in the above example, 40 features were passed to the model following the application of feature extraction and significance testing. 

Looking at the feature impact plot above, we can see that `Contract_One year` had the highest feature impact in the dataset when applied to the best model, indicating this was the most important feature when generating predictions.

#### Confusion matrix

<img src="images/run1conf.png" />

A confusion matrix is also produced within the pipeline for classification problems. We see that the final `RandomForestClassifier` model correctly classified 782 data points. 

All other outputs from this run have been stored in a directory of format `/outputs/dateTimeModels/date/run_time/`

### Test on new data

We can apply the workflow associated with our specified run to new data using the `predict` attribute returned

The function will return the target predictions for new data based on the previously fitted model and workflow.

In [ ]:
-1"Model applied to dataset:\n";
.util.printDateTimeId model1.modelInfo;
-1"\nPredictions: ";
show pred1:model1.predict[telcoInputs`xtest]
-1"\nTargets:";
show telcoInputs`ytest

We can see how the model performed using `.ml.accuracy`, used in the original default pipeline, and by producing a confusion matrix using `displayCM`:

In [ ]:
-1"Accuracy on model run using hold-out data: ",string accuracy1:.ml.accuracy[telcoInputs`ytest;pred1];

In [ ]:
.util.displayCM[value .ml.confmat[telcoInputs`ytest;pred1];`0`1;"Test Set Confusion Matrix";()];

---

## Default NLP Configurations

The function `.automl.fit` can also be applied to text data using its default configuration.

As with the example above, data must be presented with a 1-to-1 mapping between features and targets.

NLP (Natural Language Processing) specific feature extraction is then applied to any character columns within the data using NLP methods contained within the Kx [NLP Library](https://github.com/KxSystems/nlp), while normal feature extraction is applied to any remaining features.

### IMBD Dataset

The [IMBD](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) dataset contains reviews of over 50,000 movies for NLP or text analysis. The dataset consists of 2 columns, containing text reviews and the target indicating if they were positively or negatively reviewed.

### Load NLP data

Below we load in 1500 rows of the IMDB dataset.

In [ ]:
// load data
imdbData:1000#("SI";enlist ",")0:`:../data/IMBD.csv

// convert text data to string
imdbData:update string each comment from imdbData

// separate into feature and target data
imdbFeat:select comment from imdbData
imdbTarg:imdbData`tgt

// inspect data
-1"Shape of feature data is: ",(" x "sv string .ml.shape imdbFeat),"\n";
show 5#imdbFeat
-1"\nDistribution of target values:\n";
show update pcnt:.util.round[;.01]100*num%sum num from select num:count i by target from([]target:imdbTarg);

We now split the data into training and testing sets to be used with `.automl.fit` and as an independent testing set for application of the `predict` attribute.

In [ ]:
show imdbInputs:.ml.traintestsplit[imdbFeat;imdbTarg;.1]

### Inputs

The below example demonstrates a binary classification problem. Notice that this time `nlp` is being passed as the feature extraction type.

A slight modification will be made to the default parameters as this model will be saved under the name `nlpModelNotebook` and the overWriteFiles parameter will also be set to `1b` to allow users to run this notebook multiple times, overwriting the saved model each iteration.

In [ ]:
IMBDfeats   :imdbInputs`xtrain                                              / features
IMBDtarget  :imdbInputs`ytrain                                              / targets
featureType2:`nlp                                                           / NLP feature extraction
problemType2:`class                                                         / classification problem
paramDict2  :`savedModelName`overWriteFiles`seed!(`nlpModelNotebook;1b;100) / define name of model

We can now run `automl.fit` utilizing the NLP functionality.

<div class="alert alert-danger">
<b>Warning</b>: Please allow ample time for the NLP configuration to run as feature extraction and hyperparameter search can take upwards of 5 minutes to complete.
</div>

In [ ]:
start:.z.t
.automl.fit[IMBDfeats;IMBDtarget;featureType2;problemType2;paramDict2];
-1"\n.automl.fit took ",string .z.t-start;

#### Feature impact

<img src="images/run2impact.png" />

From the above example, we can see that even though one feature was passed to the model, multiple features were created using the `nlp` feature creation methods. If there was any additional non textual data present, the `normal` feature creation procedures would of been applied to them. 

Looking at the feature impact above, we can see that the features created by the `word2vec` module (`colx`) were deemed to be the most important along with various features created from the NLP spacy library

#### Confusion matrix

<img src="images/run2conf.png" />

A confusion matrix is also produced within the pipeline for classification problems. We see that the final `RandomForestClassifier` model correctly classified 129 out of 180 data points. 

All other outputs from this run have been stored in a directory of format `/outputs/namedModels/modelName/`

### Test on new data

To retrieve a model, `.automl.getModel` can be used to retrieve the metadata and associated prediction function to be used on new data from disk, either, by passing the name or the date/time of the desired model.

In [ ]:
show model2:.automl.getModel[enlist[`savedModelName]!enlist "nlpModelNotebook"]

The best model created within `automl.fit`  is applied to the unseen test data to evaluate the models performance

In [ ]:
-1"Model applied to dataset:\n";
.util.printSavedModelId model2.modelInfo;
-1"\nPredictions: ";
show pred2:model2.predict[imdbInputs`xtest]
-1"\nTargets:";
show imdbInputs`ytest

In [ ]:
-1"Accuracy on model run using hold-out data: ",string accuracy2:.ml.accuracy[imdbInputs`ytest;pred2];

In [ ]:
.util.displayCM[value .ml.confmat[imdbInputs`ytest;pred2];`0`1;"Test Set Confusion Matrix";()];

---

## Custom Configurations (Advanced)

In the previous section of the notebook, we showcased how to apply default parameters within the pipeline. In this section we will focus on how the final parameter of `.automl.fit` can be modified to apply changes to the default behaviour.

### Example 1

Below we apply `.automl.featureCreation.normal.truncSingleDecomp` to the data, this is a truncated singular value decomposition outlined [here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) and applied to all combinations of columns of type float.

A random seed of `200` will also be set.

In [ ]:
paramKeys:`seed`functions                                         // parameter names to amend
paramVals:(200;.automl.featureCreation.normal.truncSingleDecomp)  // amended values
show paramDict3:paramKeys!paramVals

In [ ]:
start:.z.t
model3:.automl.fit[telcoInputs`xtrain;telcoInputs`ytrain;`normal;`class;paramDict3]
-1"\n.automl.fit took ",string .z.t-start;

#### Feature impact
<img src="images/run3impact.png" />

We can see by looking at the feature impact that a number of the most impactful features are now derived from those generated when `.automl.featureCreation.normal.truncSingleDecomp` was applied during feature extraction, this gives some insight into the potential benefit of this form of feature extraction. 

While benefiting from increases in accuracy the addition of larger numbers of features can have the effect of slowing training time and scoring time which have have an impact in time critical use-cases.

We can now predict on the hold-out dataset in order to compare accuracy results to the default case.

In [ ]:
-1"Model applied to dataset:\n";
.util.printDateTimeId model3.modelInfo;
-1"\nPredictions: ";
show pred3:model3.predict[telcoInputs`xtest]
-1"\nTargets:";
show telcoInputs`ytest

We can see that by adding feature extraction in the normal case, we have improved the accuracy slightly increases. This is highlighted in the confusion matrix below.

In [ ]:
.util.printDateTimeId model1.modelInfo;
-1"Accuracy on default model run using held-out data: ",string[accuracy1],"\n";
.util.printDateTimeId model3.modelInfo;
-1"Accuracy on custom model run using held-out data : ",string .ml.accuracy[telcoInputs`ytest;pred3];

In [ ]:
.util.displayCM[value .ml.confmat[telcoInputs`ytest;pred3];`0`1;"Test Set Confusion Matrix";()];

### Example 2

In this example we again use the Telco dataset and highlight how to change the save options contained under `saveOption` within the parameter dictionary.

In the default case, not modified in the examples above, the system will save all outputs to disk (reports, images, config file and models). Below we will set the `saveOption` to be `0`, which means that the results will be displayed to console but nothing is persisted to disk.

Other alterations made to the `paramDict` in the below model were
1. Added a **random seed**: Here we have altered the ``` `seed``` parameter to be `175`.
2. Changed the size of the **holdout** sets to be 30% of the data at each stage.
3. Changed the **hyperparameter search** type from the default of grid search to random search and set the number of repetitions to `2`. Note that Sobol-random search is also available.

A smaller subset of 1000 datapoints will be used

In [ ]:
\S 42
features:1000?telcoFeat
target  :1000?telcoTarg

paramKeys:`saveOption`seed`holdoutSize`hyperparameterSearchType`randomSearchArgument // parameter names to amend
paramVals:(0;175;.3;`random;2)                                                       // amended values
show paramDict:paramKeys!paramVals

In [ ]:
.automl.fit[features;target;`normal;`class;paramDict];

As we can see compared to the previous default behaviour nothing has been saved down during a single run of the pipeline.

### Example 3

In this example, the IMDB dataset is used with the following changes made to the input dictionary `paramDict`:

1. **Word2vec transformation** changed from default `continuous bag of words` method to `skip-gram`.
2. **Significant feature function** changed to use the percentile based procedure.
3. **Random seed** set as `275`.

In this example, printing to screen will also be suppressed and redirected to a log file called `LogFile`

In [ ]:
.automl.updatePrinting[]   // Disable printing to screen 
.automl.updateLogging[]    // Redirect printing to log file


// new significant feature function 
.automl.newSigFeat:{[x;y]
  .ml.fresh.significantfeatures[x;y;.ml.fresh.percentile 0.10]
  }

// new parameter dictionary
paramKeys:`significantFeatures`w2v`seed`loggingFile  // parameter names to amend
paramVals:(`.automl.newSigFeat;1;275;"logFile")      // amended values
show paramDict4:paramKeys!paramVals

In [ ]:
// run automl with new parameters
model4:.automl.fit[imdbInputs`xtrain;imdbInputs`ytrain;`nlp;`class;paramDict4]

We can see that the features deemed important compared with the initial run have varied. In this iteration, very few of the features created from the NLP spacy library were deemed to be significant when predicting the target value

<img src="images/run4impact.png" />

In [ ]:
.util.printDateTimeId model4.modelInfo;
-1"\nPredictions: ";
show pred4:model4.predict[imdbInputs`xtest]
-1"\nTargets:";
show imdbInputs`ytest

Below we can see how changing the `w2v` implementation decreased the accuracy of the model compared with the initial run

In [ ]:
.util.printDateTimeId model2.modelInfo;
-1"Accuracy on default model run using held-out data: ",string[accuracy2],"\n";
.util.printDateTimeId model4.modelInfo;
-1"Accuracy on custom model run using held-out data : ",string .ml.accuracy[imdbInputs`ytest;pred4];

---